In [3]:
pip install tensorflow-gpu==2.10 keras


     ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/455.9 MB 1.5 MB/s eta 0:04:59
     ---------------------------------------- 0.2/455.9 MB 2.4 MB/s eta 0:03:13
     ---------------------------------------- 0.3/455.9 MB 2.2 MB/s eta 0:03:26
     ---------------------------------------- 0.4/455.9 MB 2.2 MB/s eta 0:03:27
     ---------------------------------------- 0.5/455.9 MB 2.2 MB/s eta 0:03:28
     ---------------------------------------- 0.6/455.9 MB 2.2 MB/s eta 0:03:28
     ---------------------------------------- 0.7/455.9 MB 2.2 MB/s eta 0:03:25
     ---------------------------------------- 0.8/455.9 MB 2.2 MB/s eta 0:03:30
     ---------------------------------------- 0.9/455.9 MB 2.2 MB/s eta 0:03:25
     ---------------------------------------- 1.0/455.9 MB 2.2 MB/s eta 0:03:24
     ---------------------------------------- 1.1/455.9 MB 2.2 MB/s eta 0:03:29
     ---------------------------------------- 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     -------------------------- ----------- 316.1/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.2/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.3/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.4/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.5/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.6/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.7/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 316.8/455.9 MB 2.2 MB/s eta 0:01:03
     -------------------------- ----------- 316.9/455.9 MB 2.2 MB/s eta 0:01:04
     -------------------------- ----------- 317.0/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 317.1/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- ----------- 317.2/455.9 MB 2.2 MB/s eta 0:01:05
     -------------------------- --------

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, Dense, LayerNormalization
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
import os
import numpy as np

In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
# Check GPU and cuDNN availability
if tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None):
    print("GPU and cuDNN are available!")
else:
    print("GPU and cuDNN are not available.")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU and cuDNN are available!


In [20]:
def load_piano_rolls(folder_path, chunk_length, num_chunks):
    pianoroll_var = []

    # Iterate through all .txt files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            piano_roll = np.loadtxt(file_path)
            print(f"Loaded {filename} with shape {piano_roll.shape}")  # Print the shape of each file

            # Create randomized chunks
            for _ in range(num_chunks):
                start_idx = np.random.randint(0, piano_roll.shape[1] - chunk_length)
                chunk = piano_roll[:, start_idx:start_idx + chunk_length]
                pianoroll_var.append(chunk)

    return np.array(pianoroll_var) # Return the chunks directly

chunk_length = 1000
num_chunks = 5  # Number of chunks to create per file

encoder_input_data = load_piano_rolls("pianoroll_pop", chunk_length, num_chunks)
decoder_target_data = load_piano_rolls("pianoroll_acapella", chunk_length, num_chunks)
decoder_input_data = np.roll(decoder_target_data, shift=1, axis=2)

print("encoder_input_data shape:", encoder_input_data.shape)
print("decoder_input_data shape:", decoder_input_data.shape)
print("decoder_target_data shape:", decoder_target_data.shape)

Loaded 1.wav.txt with shape (128, 20838)
Loaded 10.wav.txt with shape (128, 19761)
Loaded 11.wav.txt with shape (128, 19761)
Loaded 12.wav.txt with shape (128, 17860)
Loaded 13.wav.txt with shape (128, 24446)
Loaded 14.wav.txt with shape (128, 27806)
Loaded 15.wav.txt with shape (128, 21409)
Loaded 16.wav.txt with shape (128, 15295)
Loaded 17.wav.txt with shape (128, 23081)
Loaded 18.wav.txt with shape (128, 21693)
Loaded 19.wav.txt with shape (128, 17627)
Loaded 2.wav.txt with shape (128, 17183)
Loaded 20.wav.txt with shape (128, 28516)
Loaded 21.wav.txt with shape (128, 17146)
Loaded 22.wav.txt with shape (128, 14547)
Loaded 23.wav.txt with shape (128, 14547)
Loaded 24.wav.txt with shape (128, 20544)
Loaded 25.wav.txt with shape (128, 22862)
Loaded 26.wav.txt with shape (128, 22862)
Loaded 27.wav.txt with shape (128, 21063)
Loaded 28.wav.txt with shape (128, 20766)
Loaded 29.wav.txt with shape (128, 17901)
Loaded 3.wav.txt with shape (128, 22300)
Loaded 30.wav.txt with shape (128, 26

In [29]:
# Define hyperparameters
num_notes = 128
units = 64
num_layers = 2
chunk_length = 1000

# Encoder
encoder_inputs = Input(shape=(num_notes, chunk_length))
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(num_notes, chunk_length))
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(chunk_length, activation='linear'))  # Match the target size
outputs = decoder_dense(decoder_outputs)

# Compile the model
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs)
model.compile(optimizer='adam', loss='mse')  # Change loss to mse

# Summary of the model
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 128, 1000)]  0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 128, 1000)]  0           []                               
                                                                                                  
 lstm_20 (LSTM)                 [(None, 64),         272640      ['input_19[0][0]']               
                                 (None, 64),                                                      
                                 (None, 64)]                                                      
                                                                                            

In [32]:
# Train the model
history = model.fit(
    [encoder_input_data, decoder_input_data], decoder_target_data,
    epochs=1000,  # Number of epochs; adjust as needed
    batch_size=128  # Batch size; adjust as needed
)

model.save('model_saves')

Epoch 1/1000
2/2 [==============================] - 1s 78ms/step - loss: 200.3605
Epoch 2/1000
2/2 [==============================] - 0s 73ms/step - loss: 200.0697
Epoch 3/1000
2/2 [==============================] - 0s 74ms/step - loss: 199.7918
Epoch 4/1000
2/2 [==============================] - 0s 73ms/step - loss: 199.5060
Epoch 5/1000
2/2 [==============================] - 0s 71ms/step - loss: 199.2351
Epoch 6/1000
2/2 [==============================] - 0s 73ms/step - loss: 198.9593
Epoch 7/1000
2/2 [==============================] - 0s 69ms/step - loss: 198.6879
Epoch 8/1000
2/2 [==============================] - 0s 71ms/step - loss: 198.4209
Epoch 9/1000
2/2 [==============================] - 0s 70ms/step - loss: 198.1514
Epoch 10/1000
2/2 [==============================] - 0s 69ms/step - loss: 197.8854
Epoch 11/1000
2/2 [==============================] - 0s 72ms/step - loss: 197.6228
Epoch 12/1000
2/2 [==============================] - 0s 78ms/step - loss: 197.3610
Epoch 13/1000

2/2 [==============================] - 0s 75ms/step - loss: 180.8317
Epoch 100/1000
2/2 [==============================] - 0s 74ms/step - loss: 180.6933
Epoch 101/1000
2/2 [==============================] - 0s 76ms/step - loss: 180.5648
Epoch 102/1000
2/2 [==============================] - 0s 75ms/step - loss: 180.4359
Epoch 103/1000
2/2 [==============================] - 0s 72ms/step - loss: 180.2804
Epoch 104/1000
2/2 [==============================] - 0s 72ms/step - loss: 180.1475
Epoch 105/1000
2/2 [==============================] - 0s 74ms/step - loss: 180.0036
Epoch 106/1000
2/2 [==============================] - 0s 74ms/step - loss: 179.8694
Epoch 107/1000
2/2 [==============================] - 0s 70ms/step - loss: 179.7405
Epoch 108/1000
2/2 [==============================] - 0s 72ms/step - loss: 179.6113
Epoch 109/1000
2/2 [==============================] - 0s 74ms/step - loss: 179.4746
Epoch 110/1000
2/2 [==============================] - 0s 72ms/step - loss: 179.3421
Epoch 1

2/2 [==============================] - 0s 69ms/step - loss: 170.3273
Epoch 197/1000
2/2 [==============================] - 0s 75ms/step - loss: 170.2221
Epoch 198/1000
2/2 [==============================] - 0s 76ms/step - loss: 170.1350
Epoch 199/1000
2/2 [==============================] - 0s 73ms/step - loss: 170.0549
Epoch 200/1000
2/2 [==============================] - 0s 72ms/step - loss: 169.9637
Epoch 201/1000
2/2 [==============================] - 0s 73ms/step - loss: 169.8862
Epoch 202/1000
2/2 [==============================] - 0s 67ms/step - loss: 169.8030
Epoch 203/1000
2/2 [==============================] - 0s 70ms/step - loss: 169.7193
Epoch 204/1000
2/2 [==============================] - 0s 72ms/step - loss: 169.6338
Epoch 205/1000
2/2 [==============================] - 0s 72ms/step - loss: 169.5435
Epoch 206/1000
2/2 [==============================] - 0s 72ms/step - loss: 169.4610
Epoch 207/1000
2/2 [==============================] - 0s 71ms/step - loss: 169.3796
Epoch 2

2/2 [==============================] - 0s 75ms/step - loss: 163.3208
Epoch 294/1000
2/2 [==============================] - 0s 73ms/step - loss: 163.2681
Epoch 295/1000
2/2 [==============================] - 0s 72ms/step - loss: 163.1967
Epoch 296/1000
2/2 [==============================] - 0s 71ms/step - loss: 163.1610
Epoch 297/1000
2/2 [==============================] - 0s 70ms/step - loss: 163.1148
Epoch 298/1000
2/2 [==============================] - 0s 72ms/step - loss: 163.0646
Epoch 299/1000
2/2 [==============================] - 0s 70ms/step - loss: 163.0039
Epoch 300/1000
2/2 [==============================] - 0s 71ms/step - loss: 162.9309
Epoch 301/1000
2/2 [==============================] - 0s 72ms/step - loss: 162.8668
Epoch 302/1000
2/2 [==============================] - 0s 72ms/step - loss: 162.7909
Epoch 303/1000
2/2 [==============================] - 0s 68ms/step - loss: 162.7158
Epoch 304/1000
2/2 [==============================] - 0s 71ms/step - loss: 162.6535
Epoch 3

2/2 [==============================] - 0s 71ms/step - loss: 157.7657
Epoch 391/1000
2/2 [==============================] - 0s 72ms/step - loss: 157.7232
Epoch 392/1000
2/2 [==============================] - 0s 67ms/step - loss: 157.6928
Epoch 393/1000
2/2 [==============================] - 0s 68ms/step - loss: 157.6096
Epoch 394/1000
2/2 [==============================] - 0s 70ms/step - loss: 157.5427
Epoch 395/1000
2/2 [==============================] - 0s 71ms/step - loss: 157.4905
Epoch 396/1000
2/2 [==============================] - 0s 72ms/step - loss: 157.4296
Epoch 397/1000
2/2 [==============================] - 0s 73ms/step - loss: 157.4044
Epoch 398/1000
2/2 [==============================] - 0s 71ms/step - loss: 157.3504
Epoch 399/1000
2/2 [==============================] - 0s 72ms/step - loss: 157.2720
Epoch 400/1000
2/2 [==============================] - 0s 72ms/step - loss: 157.2234
Epoch 401/1000
2/2 [==============================] - 0s 72ms/step - loss: 157.1723
Epoch 4

...

2/2 [==============================] - 0s 78ms/step - loss: 139.7342
Epoch 779/1000
2/2 [==============================] - 0s 72ms/step - loss: 139.7078
Epoch 780/1000
2/2 [==============================] - 0s 71ms/step - loss: 139.7047
Epoch 781/1000
2/2 [==============================] - 0s 71ms/step - loss: 139.6698
Epoch 782/1000
2/2 [==============================] - 0s 71ms/step - loss: 139.6180
Epoch 783/1000
2/2 [==============================] - 0s 72ms/step - loss: 139.5442
Epoch 784/1000
2/2 [==============================] - 0s 75ms/step - loss: 139.4899
Epoch 785/1000
2/2 [==============================] - 0s 71ms/step - loss: 139.4425
Epoch 786/1000
2/2 [==============================] - 0s 71ms/step - loss: 139.3853
Epoch 787/1000
2/2 [==============================] - 0s 72ms/step - loss: 139.3431
Epoch 788/1000
2/2 [==============================] - 0s 72ms/step - loss: 139.2832
Epoch 789/1000
2/2 [==============================] - 0s 72ms/step - loss: 139.2279
Epoch 7

2/2 [==============================] - 0s 72ms/step - loss: 135.0639
Epoch 876/1000
2/2 [==============================] - 0s 72ms/step - loss: 135.0060
Epoch 877/1000
2/2 [==============================] - 0s 72ms/step - loss: 134.9499
Epoch 878/1000
2/2 [==============================] - 0s 72ms/step - loss: 134.9113
Epoch 879/1000
2/2 [==============================] - 0s 81ms/step - loss: 134.8626
Epoch 880/1000
2/2 [==============================] - 0s 71ms/step - loss: 134.8459
Epoch 881/1000
2/2 [==============================] - 0s 72ms/step - loss: 134.8036
Epoch 882/1000
2/2 [==============================] - 0s 70ms/step - loss: 134.7537
Epoch 883/1000
2/2 [==============================] - 0s 71ms/step - loss: 134.6953
Epoch 884/1000
2/2 [==============================] - 0s 72ms/step - loss: 134.6633
Epoch 885/1000
2/2 [==============================] - 0s 77ms/step - loss: 134.6129
Epoch 886/1000
2/2 [==============================] - 0s 69ms/step - loss: 134.5672
Epoch 8

2/2 [==============================] - 0s 71ms/step - loss: 130.4977
Epoch 973/1000
2/2 [==============================] - 0s 72ms/step - loss: 130.4426
Epoch 974/1000
2/2 [==============================] - 0s 75ms/step - loss: 130.3930
Epoch 975/1000
2/2 [==============================] - 0s 75ms/step - loss: 130.3432
Epoch 976/1000
2/2 [==============================] - 0s 72ms/step - loss: 130.2924
Epoch 977/1000
2/2 [==============================] - 0s 72ms/step - loss: 130.2468
Epoch 978/1000
2/2 [==============================] - 0s 72ms/step - loss: 130.1930
Epoch 979/1000
2/2 [==============================] - 0s 72ms/step - loss: 130.1281
Epoch 980/1000
2/2 [==============================] - 0s 76ms/step - loss: 130.0857
Epoch 981/1000
2/2 [==============================] - 0s 69ms/step - loss: 130.0424
Epoch 982/1000
2/2 [==============================] - 0s 71ms/step - loss: 129.9876
Epoch 983/1000
2/2 [==============================] - 0s 73ms/step - loss: 129.9493
Epoch 9

INFO:tensorflow:Assets written to: model_saves\assets


INFO:tensorflow:Assets written to: model_saves\assets


In [33]:
pip install mido

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Obtaining dependency information for mido from https://files.pythonhosted.org/packages/b0/0e/b551e09391220208764c00ed2b382e198414329d8b6037eeb54a8008d737/mido-1.3.0-py3-none-any.whl.metadata
  Using cached mido-1.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
Using cached mido-1.3.0-py3-none-any.whl (50 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.0
    Uninstalling packaging-23.0:
      Successfully uninstalled packaging-23.0


In [34]:
from mido import MidiFile, MidiTrack, Message

In [47]:
def pianoroll_to_midi(pianoroll, output_path, time_step_duration=480):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Threshold for note velocity
    threshold = 0.75

    # Normalize the pianoroll if needed
    max_velocity = np.max(pianoroll)
    if max_velocity > 1:
        pianoroll = pianoroll / max_velocity

    # Track the current state of the notes
    note_state = [0] * 128

    for time_step in pianoroll.T:
        for note, velocity in enumerate(time_step):
            if velocity > threshold and note_state[note] == 0:
                track.append(Message('note_on', note=note, velocity=int(velocity * 127), time=time_step_duration))
                note_state[note] = 1
            elif velocity <= threshold and note_state[note] == 1:
                track.append(Message('note_off', note=note, velocity=0, time=time_step_duration))
                note_state[note] = 0

    # Turn off any remaining notes
    for note, state in enumerate(note_state):
        if state == 1:
            track.append(Message('note_off', note=note, velocity=0, time=time_step_duration))

    mid.save(output_path)


def generate_acapella(folder_path, model, chunk_length, num_chunks):
    # Load the chunks
    encoder_input_data = load_piano_rolls(folder_path, chunk_length, num_chunks)

    # Predict each chunk
    predicted_chunks = []
    for chunk in encoder_input_data:
        # Reshape to match the model's expected input shape
        chunk = chunk.reshape((1, chunk.shape[0], chunk.shape[1]))
        prediction = model.predict([chunk, chunk])  # Assuming the model takes the same input for encoder and decoder
        predicted_chunks.append(prediction[0])

    # Combine the chunks
    combined_pianoroll = np.concatenate(predicted_chunks, axis=1)

    return combined_pianoroll

# Example usage
folder_path = "test_pianoroll_pop"
chunk_length = 20
num_chunks = 50  # Number of chunks to create per file

acapella_pianoroll = generate_acapella(folder_path, model, chunk_length, num_chunks)
pianoroll_to_midi(acapella_pianoroll, "output_acapella.mid")

Loaded 5.wav.txt with shape (128, 21034)
1/1 [==============================] - 0s 32ms/step
